# A Quantitative Analyst's Workflow for AAPL

---

## Section 1 : Why??

Sebelum kita analisis dan buat model, kita perlu ngerti dulu, kenapa kita mau prediksi harga saham AAPL??

### 1.1. Tujuan - Capital Gain & Risk Management

balik ke dasar dalam finance dan investment, invest ke capital market itu tujuannya yang gedein uang kita. nah model disini dipake untuk bantu analisis informasi atau data yang ada dengan lebih efisien dan otomatis. of course model gaakan sempurna, karena pada dasarnya sulit untuk mebemukan pola dalam human behavior penggerak ekonomi. tapi setidaknya ini bisa dijadikan bantuan dalam management investment portofolio kita.

### 1.2. Apa yang Mau di Prediksi??

Kita mau bikin model yang bisa prediksi close value dari AAPL keesokan harinya.  secara simpel itu kaya gini

$Prediksi Harga Besok = fungsi(Data Haei Ini)$

atau :

$Predicted\_Close(t+1) = f(X_t)$



Btw, kita cuma pake data sampe hari ini buat nebak harga besok. Gak boleh tau data besok buat bisa prediksi hari ini, ini disebut lookahead bias, dan ini haram hukumnya di finance modelling.

so, fitur yang kita pake untuk prediksi adalah

- harga pembukaan, tertinggi, terendah, close, dan volume (OHCLC).
- semua data diambil dari data hari hari sebelumnya, sampai hari ini.

### 1.3. Datanya dari mana?? `yfinance` jawabannya

Kita akan menggunakan data harian OHLCV (Open, High, Low, Close, Volume) untuk saham 'AAPL', menggunakan `yfinance`. kita bakal ambil data selama 5 tahun terakhir, kenapa 5 tahun?. well, 5 tahun adalah waktu yang udah cukup untuk dapet pola perubahan pasar, naik turun, big event kaya covid dan kenaikan suku bunga. dan juga ga terlalu lama yang bikin data ga relevan ikut masuk dan bikin model bias atau pusing nangkep pola.

#### Dependencies 


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from xgboost import XGBRegressor
import shap
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (15, 7)

print("Semua library berhasil di-import.")

#### Data

In [ ]:
# ticker dan periode waktu
ticker = 'AAPL'
start_date = '2019-01-01'
end_date = pd.to_datetime('today').strftime('%Y-%m-%d')

try:
    data = yf.download(ticker, start=start_date, end=end_date)
    if data.empty:
        raise ValueError("Datanya kosong. Mungkin ticker-nya salah atau gak ada data.")
    print(f"Data {ticker} berhasil diambil dari {start_date} sampai {end_date}.")
    print("Informasi umum data:")
    data.info()
except Exception as e:
    print(f"Error ambil data: {e}")

# Lihat isi data
print("\nLima baris pertama:")
display(data.head())

print("\nStatistik umum:")
display(data.describe())